In [1]:
import random
import math

# Ros libraries
import roslib
import rospy
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Range

In [2]:
PLATFORM_LENGTH_M = 0.25
PLATFORM_WIDTH_M = 0.1
ONE_TURN_RAD = math.pi

In [3]:
class SensorData:
    def __init__(self, rangeLeft, rangeFront, rangeRight):
        self._rangeLeft = rangeLeft
        self._rangeFront = rangeFront
        self._rangeRight = rangeRight

class ActuatorData:
    def __init__(self, distance, azimuth):
        self._distance = distance
        self._azimuth = azimuth

In [4]:
class ControlModel:
    def __init__(self):
        a = 1
        
    def adapt(self, SensorData):
        return ActuatorData(50*random.randint(1, 5), 50*random.randint(1, 5))

In [5]:
class ControlNode:
    def __init__(self, model):
        self._model = model
        rospy.init_node('ControlNode', anonymous=True)
        self._publisher = rospy.Publisher('/car/actuator_data', Twist, queue_size=10)
        self._subscriber = rospy.Subscriber("/car/range_left", Range, self.callback,  queue_size = 1)
    
    def actuate(self, actuator_data):
        actuatorMessage = Twist()
        actuatorMessage.linear.x = actuator_data._distance
        actuatorMessage.linear.y = 0
        actuatorMessage.linear.z = 0
        actuatorMessage.angular.x = 0
        actuatorMessage.angular.y = 0
        actuatorMessage.angular.z = actuator_data._azimuth
        self._publisher.publish(actuatorMessage)
    
    def callback(self, ros_data):       
        #time1 = time.time()
        actuatorData = self._model.adapt(SensorData(1, 1, 1))
        #time2 = time.time()
        #print("{} seconds spent to adapt", time2-time1) 
        self.actuate(actuatorData)



In [6]:
node = ControlNode(ControlModel())
try:
    rospy.spin()
except KeyboardInterrupt:
    print("Shutting down ROS control node module")